In [2]:
import polars as pl
import numpy as np
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from typing import Tuple, List
import warnings
warnings.filterwarnings('ignore')

# Convertir a DataFrame de Polars
df = pl.read_csv('CGR1.csv')

# Extraer columnas numéricas
numeric_cols = ["tmax(degC)", "tmin(degC)", "ppt(mm)", "ws(mps)", "aet(mm)", "q(mm)", "soil(mm)"]


warnings.filterwarnings('ignore')

# Función para detectar outliers usando IQR
def detect_outliers_iqr(data: np.ndarray, factor: float = 1.5) -> List[int]:
    Q1 = np.percentile(data, 25)
    Q3 = np.percentile(data, 75)
    IQR = Q3 - Q1
    lower_bound = Q1 - factor * IQR
    upper_bound = Q3 + factor * IQR
    outliers = np.where((data < lower_bound) | (data > upper_bound))[0]
    return outliers.tolist()

# Función para ajustar distribución y calcular K-S
def fit_distribution(data: np.ndarray, distributions: List[str]) -> Tuple[str, float, float, tuple]:
    best_dist = None
    best_ks_stat = float('inf')
    best_p_value = 0
    best_params = None
    max_p_value = 0
    for dist_name in distributions:
        try:
            dist = getattr(stats, dist_name)
            params = dist.fit(data)
            ks_stat, p_value = stats.kstest(data, dist_name, args=params)
            if p_value > max_p_value and ks_stat < best_ks_stat:
                best_dist = dist_name
                best_ks_stat = ks_stat
                best_p_value = p_value
                best_params = params
        except Exception:
            continue

    return best_dist, best_ks_stat, best_p_value, best_params

# Función para generar histograma y CDF
def plot_histogram_and_cdf(data: np.ndarray, col_name: str, dist_name: str, params: tuple):
    fig = make_subplots(rows=2, cols=1, subplot_titles=["Histograma", "CDF Empírica vs Teórica"])

    # Histograma
    hist_data = data
    hist_fig = px.histogram(hist_data, nbins=30, title=f"Histograma de {col_name}")
    hist_fig.update_layout(height=300, showlegend=False)

    # CDF
    x = np.linspace(min(data), max(data), 1000)
    cdf_emp = np.sort(data)
    cdf_emp = np.interp(x, np.sort(data), np.linspace(0, 1, len(data)))
    # Crear la CDF teórica
    dist = getattr(stats, dist_name)
    cdf_theo = dist.cdf(x, *params)

    cdf_fig = go.Figure()
    cdf_fig.add_trace(go.Scatter(x=x, y=cdf_theo, mode='lines', name='CDF Teórica', line=dict(color='blue')))
    cdf_fig.add_trace(go.Scatter(x=x, y=cdf_emp, mode='lines', name='CDF Empírica', line=dict(color='red')))
    cdf_fig.update_layout(title=f"Comparación CDF: {col_name}", xaxis_title="Valor", yaxis_title="CDF")

    fig.add_trace(hist_fig.data[0], row=1, col=1)
    fig.add_trace(cdf_fig.data[0], row=2, col=1)

    fig.update_layout(title_text=f"Análisis de {col_name}", height=600)
    fig.show()

# Función para análisis estadístico básico
def basic_stats(data: np.ndarray, col_name: str):
    stats_dict = {
        "Media": np.mean(data),
        "Mediana": np.median(data),
        "Desviación estándar": np.std(data),
        "CV": np.std(data) / np.mean(data) if np.mean(data) != 0 else 0,
        "Mínimo": np.min(data),
        "Máximo": np.max(data),
        "Q1": np.percentile(data, 25),
        "Q3": np.percentile(data, 75),
        "IQR": np.percentile(data, 75) - np.percentile(data, 25),
        "Outliers": detect_outliers_iqr(data),
        "Longitud": len(data)
    }
    return stats_dict

# Función principal de análisis por columna
def analyze_column(df: pl.DataFrame, col_name: str):
    data = df[col_name].to_numpy()
    stats = basic_stats(data, col_name)
    print(f"\n=== Análisis de {col_name} ===")
    for key, value in stats.items():
        print(f"{key}: {value:.4f}" if isinstance(value, (int, float)) else f"{key}: {value}")

    # Detectar outliers
    outliers = stats["Outliers"]
    print(f"\nOutliers detectados en {col_name}: {len(outliers)} ({outliers})")

    # Ajuste de distribución
    distributions = [
        "norm", "expon", "gamma", "beta", "weibull_min", "lognorm",
        "chi2", "t", "f", "laplace", "cauchy", "logistic", "gumbel_r", "gumbel_l"
    ]
    dist_name, ks_stat, p_value, params = fit_distribution(data, distributions)
    print(f"\nMejor distribución ajustada: {dist_name}")
    print(f"Estadístico K-S: {ks_stat:.4f}, p-valor: {p_value:.4f}")

    # Visualización
    plot_histogram_and_cdf(data, col_name, dist_name, params)

# Ejecutar análisis por cada columna
for col in numeric_cols:
    analyze_column(df, col)

# Fin del análisis
print("\n✅ Análisis completado para todas las columnas.")


=== Análisis de tmax(degC) ===
Media: 29.4359
Mediana: 29.9000
Desviación estándar: 1.8589
CV: 0.0632
Mínimo: 23.3500
Máximo: 33.5900
Q1: 28.0850
Q3: 30.8100
IQR: 2.7250
Outliers: [54, 161, 414]
Longitud: 780.0000

Outliers detectados en tmax(degC): 3 ([54, 161, 414])

Mejor distribución ajustada: gumbel_l
Estadístico K-S: 0.0534, p-valor: 0.0227



=== Análisis de tmin(degC) ===
Media: 18.3147
Mediana: 18.9700
Desviación estándar: 2.3446
CV: 0.1280
Mínimo: 11.0900
Máximo: 23.0500
Q1: 16.7050
Q3: 20.1725
IQR: 3.4675
Outliers: [54, 414, 510]
Longitud: 780.0000

Outliers detectados en tmin(degC): 3 ([54, 414, 510])

Mejor distribución ajustada: gumbel_l
Estadístico K-S: 0.0624, p-valor: 0.0044



=== Análisis de ppt(mm) ===
Media: 119.5021
Mediana: 107.5000
Desviación estándar: 81.9806
CV: 0.6860
Mínimo: 0.4000
Máximo: 446.1000
Q1: 56.8500
Q3: 171.3000
IQR: 114.4500
Outliers: [108, 169, 228, 372, 444, 468, 540, 564, 587, 588, 600, 623, 695]
Longitud: 780.0000

Outliers detectados en ppt(mm): 13 ([108, 169, 228, 372, 444, 468, 540, 564, 587, 588, 600, 623, 695])

Mejor distribución ajustada: lognorm
Estadístico K-S: 0.0428, p-valor: 0.1111



=== Análisis de ws(mps) ===
Media: 2.8273
Mediana: 2.8100
Desviación estándar: 0.5115
CV: 0.1809
Mínimo: 1.5100
Máximo: 4.3000
Q1: 2.4900
Q3: 3.2000
IQR: 0.7100
Outliers: [126]
Longitud: 780.0000

Outliers detectados en ws(mps): 1 ([126])

Mejor distribución ajustada: t
Estadístico K-S: 0.0268, p-valor: 0.6185



=== Análisis de aet(mm) ===
Media: 89.2565
Mediana: 102.0000
Desviación estándar: 37.5874
CV: 0.4211
Mínimo: 1.7000
Máximo: 154.0000
Q1: 64.2000
Q3: 117.3500
IQR: 53.1500
Outliers: []
Longitud: 780.0000

Outliers detectados en aet(mm): 0 ([])

Mejor distribución ajustada: beta
Estadístico K-S: 0.0761, p-valor: 0.0002



=== Análisis de q(mm) ===
Media: 30.2924
Mediana: 5.4000
Desviación estándar: 52.3932
CV: 1.7296
Mínimo: 0.0000
Máximo: 348.0000
Q1: 2.8000
Q3: 33.2750
IQR: 30.4750
Outliers: [0, 1, 11, 12, 13, 24, 25, 37, 39, 59, 60, 61, 73, 83, 84, 85, 95, 108, 109, 120, 131, 132, 156, 157, 169, 180, 191, 192, 194, 203, 207, 215, 217, 227, 228, 251, 252, 263, 264, 265, 275, 276, 287, 289, 290, 299, 300, 303, 304, 312, 323, 324, 337, 348, 359, 361, 372, 373, 384, 388, 407, 409, 410, 411, 412, 420, 443, 444, 445, 455, 456, 457, 468, 469, 479, 480, 481, 482, 491, 505, 506, 516, 517, 527, 529, 540, 541, 552, 556, 564, 576, 577, 587, 588, 589, 600, 601, 612, 623, 624, 625, 636, 637, 648, 649, 672, 673, 684, 695, 696, 697, 708, 709, 720, 721]
Longitud: 780.0000

Outliers detectados en q(mm): 115 ([0, 1, 11, 12, 13, 24, 25, 37, 39, 59, 60, 61, 73, 83, 84, 85, 95, 108, 109, 120, 131, 132, 156, 157, 169, 180, 191, 192, 194, 203, 207, 215, 217, 227, 228, 251, 252, 263, 264, 265, 275, 276, 287, 289, 290, 299, 


=== Análisis de soil(mm) ===
Media: 22.9512
Mediana: 15.6000
Desviación estándar: 15.6389
CV: 0.6814
Mínimo: 3.3000
Máximo: 42.3000
Q1: 8.2750
Q3: 42.3000
IQR: 34.0250
Outliers: []
Longitud: 780.0000

Outliers detectados en soil(mm): 0 ([])

Mejor distribución ajustada: lognorm
Estadístico K-S: 0.1891, p-valor: 0.0000



✅ Análisis completado para todas las columnas.
